In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
from dataclasses import dataclass
from typing import *
import logging

import pandas as pd

import lass.test
import lass.train
import lass.datasets
import lass.pipeline
from lass.log_handling import LogLoader, LogLoaderArgs, PaperTasks

In [3]:
@dataclass
class Architecture():
    name: str
    name_short: str
    batch_size: int
    gradient_accumulation_steps: int
    
architecture = Architecture(
    name="microsoft/deberta-v3-base", 
    name_short="deberta-base",
    batch_size=16,
    gradient_accumulation_steps=2,
)


log_loader_args = LogLoaderArgs(
    logdir='../artifacts/logs/',
    tasks='paper-full',
    model_families=['BIG-G T=0'],
    model_sizes=['4b', '8b', '27b', '128b'],
    shots=[0],
    query_types=['multiple_choice'],
)

# model = lass.train.train(
#     data_args=log_loader_args,
#     group='population',
#     output_dir="population",
#     split='instance',
#     model_name=architecture.name,
#     model_name_short=f"{architecture.name_short}-pop-top4",
#     batch_size=architecture.batch_size,
#     gradient_accumulation_steps=architecture.gradient_accumulation_steps,
#     include_model_in_input=True,
#     include_n_targets_in_input=False,
#     n_epochs=6,
#     test_max_instances=5000,
#     extra_training_args={
#         "warmup_steps": 3000,
#         "learning_rate": 2e-5,
#         'eval_steps': 2000,
#         'save_steps': 2000,
#         'logging_steps': 2000,
#     },
#     # is_test_run=True,
# )

# NOTE: Requires temporary hack in lass/test.py to test on 128b only
# model_loc = "../artifacts/assessors/population/deberta-base-pop-top4-bs16*2-0sh-pop-instance-split-01191527/checkpoint-21500"
model_loc = "./population/deberta-base-pop-top4-bs16*2-0sh-pop-instance-split-01201417/checkpoint-6000"
info: dict = lass.test.test(
    data_args=log_loader_args,
    split='instance',
    model_loc=model_loc,
    model_name=architecture.name,
    per_task=True,
) # type: ignore


task_results = {task: info["tasks"][task]["metrics"] | {'count': len(info["tasks"][task]["test"])} for task in info["tasks"].keys()}
task_results['_total'] = info['metrics'] | {'count': len(info['test'])}

df = pd.DataFrame.from_dict(task_results, orient='index')
df.to_csv(f"population/results-base-top4-128b-only.csv")

/home/wout/pp/lass/src/lass/pipeline.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'correct'] = df['correct'].astype(int)


{'metrics': {'conf-absolute': {'acc': {'test': 0.6538817005545287,
                                       'train': 0.6710337390943322},
                               'balanced_acc': {'test': 0.5045639938807116,
                                                'train': 0.5014413440483553},
                               'bs': {'test': 0.3074915108549931,
                                      'train': 0.30287469460759775},
                               'bs_dcr': {'test': 0.000751703079888949,
                                          'train': 0.00018496065261627836},
                               'bs_mcb': {'test': 0.08289975979880898,
                                          'train': 0.08297072737300726},
                               'bs_unc': {'test': 0.22534345413607307,
                                          'train': 0.22008892788720677},
                               'roc_auc': {'test': 0.4922418358410946,
                                           'train': 0.48247211738810

/home/wout/pp/lass/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


loading configuration file https://huggingface.co/microsoft/deberta-v3-base/resolve/main/config.json from cache at /home/wout/.cache/huggingface/transformers/e6f9db57345f0f60c9f837fa97bcb27b1ed31e99feb33d732d7d8c80cb8f8459.de97182a9f32a68819030ba8f3f6ff2ba47276be3864425925523202f54cc79c


Model config DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.3",
  "type_vocab_size": 0,
  "vocab_size": 128100
}



...

  0%|          | 0/11 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.


***** Running Prediction *****


  Num examples = 10820


  Batch size = 8
